<a href="https://colab.research.google.com/github/SinghReena/MachineLearning/blob/master/7_GroupBy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

## Motivation for GroupBy

Suppose we have a dataset that can be grouped by many attributes. We want to compute statistics on the various subsets based on groups. How can we do it efficiently?

Here's a dataset with 9 students, their gender and year is given.  What if we want to compute the average of each gender?  Or based on year?  Or based on both?  We should subset exhaustively by hand.

In [ ]:
df = pd.DataFrame({'name' : ['Alice', 'Bob', 'Charlie', 'Daniele', 'Eva', 'Frank', 'Gus', 'Harriet', 'Ivan'],
                   'gender' : ['F', 'M', 'M', 'F', 'F', 'M', 'M', 'F', 'M'],
                   'year' : ['junior', 'senior', 'junior', 'senior', 'junior', 'senior', 'junior', 'senior', 'junior'],
                   'data1' : np.random.randint(0, 10,9),
                   'data2' : np.random.randint(11, 19, 9)})
df

,name,gender,year,data1,data2
0,Alice,F,junior,6,14
1,Bob,M,senior,7,12
2,Charlie,M,junior,5,13
3,Daniele,F,senior,6,14
4,Eva,F,junior,0,14
5,Frank,M,senior,2,18
6,Gus,M,junior,4,15
7,Harriet,F,senior,4,13
8,Ivan,M,junior,0,12


In [ ]:
d2 = df.data2

In [ ]:
d2.iloc[1] = 27


/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [ ]:
df


,name,gender,year,data1,data2
0,Alice,F,junior,8,12
1,Bob,M,senior,0,27
2,Charlie,M,junior,7,13
3,Daniele,F,senior,3,13
4,Eva,F,junior,6,12
5,Frank,M,senior,6,12
6,Gus,M,junior,1,14
7,Harriet,F,senior,3,14
8,Ivan,M,junior,8,11


To compute mean scores based on gender, first compute the average for `M`, then for `F`.

In [ ]:
df[df['gender'] == 'M'].mean()

data1     6.2
data2    16.2
dtype: float64

In [ ]:
df[df['gender'] == 'F'].mean()

data1     5.50
data2    15.25
dtype: float64

When there are more then two groups we have to repeat the process multiple times. Each of these has to be computed by hand and stored separately. This task is tedious and error-prone.  Further computations, example plotting, will be hard. What if we could do this automatically?  

Enter GroupBy.

## GroupBy and Split-Apply-Combine

GroupBy in Pandas is a way to group the data into subsets to which we can apply an operation. We will apply one or more of the following steps.

- **Splitting** the data into groups
- **Apply** a function on the groups
- **Combine** the results in a datastructure.

This is a powerful concept.  We can use the apply function for many tasks. Some examples:
- We can compute sample statistics --- mean, median, count, standard deviation etc. on these groups.
- Fill the missing values in each group separately based on the mean of the group, rather than the entire data.
- Filter groups based on some criteria.  Outlier groups based on average, size etc.



**Analogy to SQL**

SQL has a `GROUPBY`.

```
SELECT GENDER, YEAR, mean(data1), sum(data2)
FROM ScoresTable
GROUP BY GENDER, YEAR
ORDER BY GENDER, YEAR;
```


A simple groupby to list means of the two columns, grouped by `gender`.

In [ ]:
df.groupby(['gender']).mean()

,data1,data2
gender,,
F,4.75,15.75
M,4.60,14.60


In [ ]:
df.groupby(['gender'])['name'].count()

gender
F    4
M    5
Name: name, dtype: int64

### Groupby with multiple columns

This creates a hierarchical clustering.

In [ ]:
df.groupby(['gender', 'year']).mean()

data1      data2
gender year                       
F      junior  4.000000  15.000000
       senior  5.500000  16.500000
M      junior  1.666667  14.666667
       senior  9.000000  14.500000

We can also reverse the order to get a different hierarchy.  (But the computated values of the corresponding groups are the same.)

In [ ]:
df.groupby(['year', 'gender']).mean()

data1      data2
year   gender                     
junior F       4.000000  15.000000
       M       1.666667  14.666667
senior F       5.500000  16.500000
       M       9.000000  14.500000

In [ ]:
df.groupby(['gender', 'year']).mean()

data1      data2
gender year                       
F      junior  4.000000  15.000000
       senior  5.500000  16.500000
M      junior  1.666667  14.666667
       senior  9.000000  14.500000

### Subset only select columns

In [ ]:
df.groupby(['gender', 'year'])['data1'].mean()

gender  year  
F       junior    8.000000
        senior    3.000000
M       junior    6.666667
        senior    5.500000
Name: data1, dtype: float64

Another way to do the subset columns.

In [ ]:
grouped = df['data2'].groupby(df['gender'])
grouped.mean()

gender
F    15.25
M    16.20
Name: data2, dtype: float64

###More aggregate functions using the `aggregate` method.

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.aggregate.html

In [ ]:
df.groupby(['gender'])['data1'].aggregate(["mean", "min"])

,mean,min
gender,,
F,5.5,0
M,6.2,3


## GroupBy internals

In [ ]:
grouped = df.groupby('gender')
grouped

In [ ]:
grouped.ngroups

2

In [ ]:
grouped.groups

{'F': [0, 3, 4, 7], 'M': [1, 2, 5, 6, 8]}

Notice that the values are not from `data2`, but it only contains the indices. This is because we are grouping based on the gender column only. So we can iterate over the groups and use the index position to get more data from the dataframe.

In [ ]:
for key, value in grouped.groups.items():
  print("key:", key)
  print("values", df.iloc[value])

key: F
values   gender    year     name  data1  data2
0      F  junior    Alice      4     15
3      F  senior  Daniele      7     17
4      F  junior      Eva      4     15
7      F  senior  Harriet      4     16
key: M
values   gender    year     name  data1  data2
1      M  senior      Bob      9     15
2      M  junior  Charlie      2     17
5      M  senior    Frank      9     14
6      M  junior      Gus      1     15
8      M  junior     Ivan      2     12


### `Unstack` to get a table

In [ ]:
data1_means = df.groupby(['year', 'gender'])['data1'].mean()

In [ ]:
data1_means

year    gender
junior  F         8.000000
        M         6.666667
senior  F         3.000000
        M         5.500000
Name: data1, dtype: float64

In [ ]:
data1_means.unstack()

gender,F,M
year,,
junior,8.0,6.666667
senior,3.0,5.500000


## Exercises for the Berkeley dataset

1. How many events (of any kind) have occured on different days of the week?
2. `CVLEGEND` has 21 kinds of crimes. How many events (`CVLEGEND`) of each kind have occured?
3. Pick the crime type that is most common.  How many events of this kind occur on different days of the week?
4. How many events of any kind have happened on different days of the week? (this can have upto 21x7 groups for 21 crime types and 7 days of the week.)
5. Last week we created `offense_df`. This was a subset of `CVLEGEND` and `OFFENSE` columns. There are 28 `OFFENSE` types and 21 `CVLEGEND` types.  How many offenses of each kind are listed per `CVLEGEND` type?

## Pivot Tables


When we group on two columns, we get two levels of indices.  This is a "tall", "skinny" table.  We can instead get a "wide" table.  Pivot table is one way to reshape the table.

In [ ]:
df

,name,gender,year,data1,data2
0,Alice,F,junior,6,14
1,Bob,M,senior,7,12
2,Charlie,M,junior,5,13
3,Daniele,F,senior,6,14
4,Eva,F,junior,0,14
5,Frank,M,senior,2,18
6,Gus,M,junior,4,15
7,Harriet,F,senior,4,13
8,Ivan,M,junior,0,12


In [ ]:
# summarizing the data by two groups.

data1_means = df.groupby(['year', 'gender'])['data1'].mean()
data1_means.to_frame()

data1
year   gender       
junior F         3.0
       M         3.0
senior F         5.0
       M         4.5

In [ ]:
df.pivot_table(values="data1",
               index = "gender", 
               columns="year",aggfunc=np.mean)

year,junior,senior
gender,,
F,3.0,5.0
M,3.0,4.5


There are a few parameters that we can use to shape the pivot table.
- `values` for data to summarize 
- `columns` for the columns.
- `index` for the row.
- `aggfunc` for the aggregation function.
- `margins` for the sum over the rows/columns

In [ ]:
df.pivot_table(values= ("data1","data2"),
               index = "gender", 
               columns="year",
               aggfunc=np.mean)

data1             data2       
year   junior senior     junior senior
gender                                
F         3.0    5.0  14.000000   13.5
M         3.0    4.5  13.333333   15.0

Pivot table is the same as `unstack` we discussed earlier. Unstacking can be done at different levels.

In [ ]:
data1_means.index

MultiIndex([('junior', 'F'),
            ('junior', 'M'),
            ('senior', 'F'),
            ('senior', 'M')],
           names=['year', 'gender'])

In [ ]:
df.groupby(["gender", "year"])["data1"].mean().unstack()

year,junior,senior
gender,,
F,2.000000,0.5
M,3.666667,9.0


In [ ]:
df.groupby(["gender", "year"])["data1"].mean().unstack(0)

gender,F,M
year,,
junior,3.0,3.0
senior,5.0,4.5


### Application in the Berkeley dataset.

Summarize the count of crime events for each type of crime for each day of the week.

There are 7 days of the week and 21 types of crimes.  A two-level index would give us 7x21 rows.  Pivoting is a better way of summarizing the data.

## Application to SF Building permits dataset.

## `apply` methods



### Lambdas in python

Lambdas are a way to encode short functions.  Lambdas are anonymous functions that do not store internal state. Lambdas are a succinct way to represent an expression based on some parameters. 

In [ ]:
increment = lambda x: x+1
increment(5.5)

6.5

lambdas are anonymous, so we will either have to assign it to a variable or pass it to a function that accepts function objects as parameters.

Syntactically, lambdas are different from functions in that they do not have the `def` keyword or a `return` statement. Lambdas do not have an internal variables and can only be a single expression.

Lambdas can have conditionals and take multiple arguments.

In [ ]:
# A lambda can have conditionals
foo = lambda x: "Pass" if x > 5 else "Fail"
foo(5)

'Fail'

In [ ]:
bar = lambda x: "fail" if x < 5 else ("good" if x > 6 else "pass")
print(bar(4))
print(bar(5))
print(bar(7))



fail
pass
good


In [ ]:
# no assignment to a variable.  Called when it is defined.
(lambda a, b: a**b)(2, 5)

32

Lambdas are most useful when we have to pass a function object as a parameter.

In [ ]:
li = [5, 10, 15, 20, 25, 30, 35] 
list(filter(lambda x: x%2 == 0, li))

[10, 20, 30]

In [ ]:
list(map(lambda x: x**2, range(10)))

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]

## Lambdas as a argument to pandas `apply`

Recall the `map` method we had applied to every entry of the dataframe.

In [ ]:
df["gender"]

0    F
1    M
2    M
3    F
4    F
5    M
6    M
7    F
8    M
Name: gender, dtype: object

In [ ]:
df["gender"].map({"F": "Female", "M": "Male"})

0    Female
1      Male
2      Male
3    Female
4    Female
5      Male
6      Male
7    Female
8      Male
Name: gender, dtype: object

We can apply a function instead of a simple table look up using `apply` method.

In [ ]:
df["data2"].apply(lambda x: x/2)

0    7.0
1    6.0
2    6.5
3    7.0
4    7.0
5    9.0
6    7.5
7    6.5
8    6.0
Name: data2, dtype: float64

This is similar to broadcasting.  But not all functions can be "broadcast", we will have to use the `apply` function.

In [ ]:
df["data2"]/2

0    7.0
1    6.0
2    6.5
3    7.0
4    7.0
5    9.0
6    7.5
7    6.5
8    6.0
Name: data2, dtype: float64

In [ ]:
df["data1"]

0    6
1    7
2    5
3    6
4    0
5    2
6    4
7    4
8    0
Name: data1, dtype: int64

In [ ]:
df["data1"].apply(lambda x: "Pass" if x > 4 else "Fail")

0    Pass
1    Pass
2    Pass
3    Pass
4    Fail
5    Fail
6    Fail
7    Fail
8    Fail
Name: data1, dtype: object

# New Section

In [ ]:
df["grade1"] = df["data1"].apply(lambda x: "Pass" if x > 0 else "Fail")

In [ ]:
df

,name,gender,year,data1,data2,grade1
0,Alice,F,junior,6,14,Pass
1,Bob,M,senior,7,12,Pass
2,Charlie,M,junior,5,13,Pass
3,Daniele,F,senior,6,14,Pass
4,Eva,F,junior,0,14,Fail
5,Frank,M,senior,2,18,Pass
6,Gus,M,junior,4,15,Pass
7,Harriet,F,senior,4,13,Pass
8,Ivan,M,junior,0,12,Fail
